# Covid-19 India Visualization and Animation

## Description

Coronavirus disease 2019 (COVID-19) is an infectious disease caused by severe acute respiratory syndrome coronavirus 2 (SARS-CoV-2).[6] The disease was first identified in December 2019 in Wuhan, the capital of China's Hubei province, and has since spread globally, resulting in the ongoing 2019–20 coronavirus pandemic.[7][8] Common symptoms include fever, cough, and shortness of breath.[9] Other symptoms may include muscle pain, sputum production, diarrhea, sore throat, loss of smell, and abdominal pain.[4][10][11] While the majority of cases result in mild symptoms, some progress to viral pneumonia and multi-organ failure.[7][12] As of 3 April 2020, more than 1,040,000[5] cases of COVID-19 have been reported in more than two hundred countries and territories,[13] resulting in over 55,100 deaths.[5] More than 221,000 people have recovered.[5]

The virus is mainly spread during close contact,[a] and by small droplets produced when people cough, sneeze, or talk.[14][16][15] Respiratory droplets may be produced during breathing but the virus is not generally airborne.[14][16] People may also contract COVID-19 by touching a contaminated surface and then their face.[14][15] It is most contagious when people are symptomatic, although spread may be possible before symptoms appear.[15] The virus can survive on surfaces up to 72 hours.[17] Time from exposure to onset of symptoms is generally between two and fourteen days, with an average of five days.[9][18] The standard method of diagnosis is by reverse transcription polymerase chain reaction (rRT-PCR) from a nasopharyngeal swab.[19] The infection can also be diagnosed from a combination of symptoms, risk factors and a chest CT scan showing features of pneumonia.[20][21]

Recommended measures to prevent infection include frequent hand washing, social distancing (maintaining physical distance from others, especially from those with symptoms), covering coughs and sneezes with a tissue or inner elbow, and keeping unwashed hands away from the face.[22][23] The use of masks is recommended for those who suspect they have the virus and their caregivers.[24] Recommendations for mask use by the general public vary, with some authorities recommending against their use, some recommending their use, and others requiring their use.[25][26][27] Currently, there is no vaccine or specific antiviral treatment for COVID-19. Management involves treatment of symptoms, supportive care, isolation, and experimental measures.[28]

The World Health Organization (WHO) declared the 2019–20 coronavirus outbreak a Public Health Emergency of International Concern (PHEIC)[29][30] on 30 January 2020, and a pandemic on 11 March 2020.[8] Local transmission of the disease has been recorded in many countries across all six WHO regions.[31] 

Source https://en.wikipedia.org/wiki/Coronavirus_disease_2019

In [1]:
# import libs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

import plotly.express as px
import plotly.graph_objects as go

# Load covid_19_india
covid_19_india_df = pd.read_csv('./data/covid_19_india.csv', index_col='Sno')

covid_19_india_df.Date = pd.to_datetime(covid_19_india_df.Date, format='%d/%m/%y')
covid_19_india_df = covid_19_india_df.dropna()

## All cases

In [2]:
date_table = covid_19_india_df.groupby(['Date', 'State/UnionTerritory']).sum().reset_index().set_index('Date')
total = date_table.loc[date_table.last_valid_index()].sum()
confirmed_count = int(total.Confirmed)
death_count = int(total.Deaths)
cured_count = int(total.Cured)

html_template = "<span style='color:{}; font-size:1.4em;'>{}</span>"
cured = html_template.format('green', 'Cured - '+str(cured_count))    
confirmed = html_template.format('blue', 'Confirmed cases - '+str(confirmed_count))    
deaths = html_template.format('red', 'Deaths - '+str(death_count))

display(Markdown(html_template.format('black', 'Summary :')))
display(Markdown(cured))
display(Markdown(confirmed))
display(Markdown(deaths))

<span style='color:black; font-size:1.4em;'>Summary :</span>

<span style='color:green; font-size:1.4em;'>Cured - 1344</span>

<span style='color:blue; font-size:1.4em;'>Confirmed cases - 11933</span>

<span style='color:red; font-size:1.4em;'>Deaths - 392</span>

In [3]:
today_states = date_table.loc[date_table.last_valid_index()].reset_index()
max_confrim = today_states[today_states.Confirmed == today_states.Confirmed.max()]
max_deaths = today_states[today_states.Deaths == today_states.Deaths.max()]
max_cured = today_states[today_states.Cured == today_states.Cured.max()]

display(Markdown(html_template.format('black', 'States summary :')))
display(Markdown(html_template.format('DodgerBlue','Maximum confirmed cases - '+ str(max_confrim.Confirmed.iloc[0]) + ', '+max_confrim['State/UnionTerritory'].iloc[0])))
display(Markdown(html_template.format('MediumSeaGreen','Maximum cured - '+ str(max_cured.Cured.iloc[0]) + ', '+max_cured['State/UnionTerritory'].iloc[0])))
display(Markdown(html_template.format('Tomato','Maximum deaths - '+ str(max_deaths.Deaths.iloc[0]) + ', '+max_deaths['State/UnionTerritory'].iloc[0])))

<span style='color:black; font-size:1.4em;'>States summary :</span>

<span style='color:DodgerBlue; font-size:1.4em;'>Maximum confirmed cases - 2687, Maharashtra</span>

<span style='color:MediumSeaGreen; font-size:1.4em;'>Maximum cured - 259, Maharashtra</span>

<span style='color:Tomato; font-size:1.4em;'>Maximum deaths - 178, Maharashtra</span>

In [4]:
date_group = covid_19_india_df.groupby(['Date']).sum()
date_group.reset_index(inplace=True)

date_group.sort_values('Date',inplace=True)

from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=3, shared_yaxes=True,
                    subplot_titles=("Total deaths - " + str(death_count),"Total confirmed - " + str(confirmed_count),
                                    "Total cured - " + str(cured_count)))

# Create and style traces
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Deaths, name='Deaths',
                         line=dict(color='firebrick', width=1), mode='lines+markers'), row=1, col=1)
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Confirmed, name = 'Confirmed',
                         line=dict(color='royalblue', width=1), mode='lines+markers',), row=1, col=2)
fig.add_trace(go.Scatter(x=date_group.Date, y=date_group.Cured, name='Cured',
                         line=dict(color='green', width=1), mode='lines+markers',), row=1, col=3)

fig.update_layout(
    title="All Cases",
    xaxis_title="Date",
    yaxis_title="Count",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.update_xaxes(ticks="inside")
fig.update_yaxes(ticks="inside", col=1)

fig.show()



In [5]:
fig = go.Figure(data=[
    go.Bar(name='Confirmed', x=date_group.Date, y=date_group.Confirmed),
    go.Bar(name='Deaths', x=date_group.Date, y=date_group.Deaths),
    go.Bar(name='Cured', x=date_group.Date, y=date_group.Cured)
])
# Change the bar mode
fig.update_layout(autosize=False,
    width=1000,
    height=750,
    title="All Cases",
    xaxis_title="Date",
    yaxis_title="Count",
    barmode='relative')
fig.show()

## Confirmed cases

In [6]:


import plotly.express as px
fig = px.bar(date_group, x='Date', y='Confirmed', color='Confirmed')

fig.update_layout(
    title="Confirmed cases",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

## Deaths

In [7]:
fig = px.bar(date_group, x='Date', y='Deaths', color='Deaths')

fig.update_layout(
    title="Deaths",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

## Cured

In [8]:
fig = px.bar(date_group, x='Date', y='Cured', color='Cured')

fig.update_layout(
    title="Cured trend",    
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="black"
    )
)

fig.show()

## State wise trend

In [9]:
fig = go.Figure(data=[
    go.Bar(name='Confirmed', x=date_table.loc[date_table.last_valid_index()]['State/UnionTerritory'], y=date_table.loc[date_table.last_valid_index()].Confirmed),
    go.Bar(name='Deaths', x=date_table.loc[date_table.last_valid_index()]['State/UnionTerritory'], y=date_table.loc[date_table.last_valid_index()].Deaths),
    go.Bar(name='Cured', x=date_table.loc[date_table.last_valid_index()]['State/UnionTerritory'], y=date_table.loc[date_table.last_valid_index()].Cured)
])
# Change the bar mode
fig.update_layout(
    autosize=False,
    width=1000,
    height=750,
    title="State wise cases",
    xaxis_title="State",
    yaxis_title="Count",
    barmode='stack')
fig.show()

### Confirmed cases pie chart

In [10]:
fig = px.pie(date_table.loc[date_table.last_valid_index()], values='Confirmed', names='State/UnionTerritory', title='Confirmed cases')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Deaths pie chart

In [11]:
fig = px.pie(date_table.loc[date_table.last_valid_index()], values='Deaths', names='State/UnionTerritory', title='Deaths')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### Cured pie chart

In [12]:
fig = px.pie(date_table.loc[date_table.last_valid_index()], values='Cured', names='State/UnionTerritory', title='Cured')
fig.update_layout(
    autosize=False,
    width=900,
    height=650)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### State wise confirmed cases animation

In [13]:
dummy = pd.DataFrame()

dummy['State'] = covid_19_india_df['State/UnionTerritory'].unique()

#colors = sns.dark_palette("purple", 33).as_hex()
#colors = sns.diverging_palette(250, 15, s=75, l=40, n=33, center="dark").as_hex()
#colors = sns.color_palette("Blues_d", 33).as_hex()
colors =sns.color_palette("hls", 33).as_hex()
def animate_bar_chart():
    frames = []
    grouped = covid_19_india_df[covid_19_india_df['Date'] > '2020/03'][['Date', 'State/UnionTerritory', 'Confirmed', 'Deaths', 'Cured']].groupby(['Date'])    
    for name, group in iter(grouped):
        merged = pd.merge(group, dummy, how='outer', left_on='State/UnionTerritory', right_on='State')        
        merged.fillna(0, inplace=True)
        merged.sort_values('State', inplace=True)
        frames.append(go.Frame(data = [go.Bar(x = merged['State'].tolist(), y=merged['Confirmed'].tolist(), marker_color=colors)], 
                              layout=go.Layout(title='Confirmed cases - '+group.Date.iloc[0].strftime('%Y-%m-%d'))))
    
    fig = go.Figure(
        data = [go.Bar(x = merged['State'].tolist(), y = [0] * len(merged['State'].tolist()))],
        frames=frames, 
        layout=go.Layout(
            width=1000,
            height=750,
            xaxis=dict(type='category'),
            yaxis=dict(range=[0, 3000], autorange=False),            
            title="Confirmed cases",
            xaxis_title="State",
            yaxis_title="Count",
            updatemenus=[dict(
                type="buttons",
                buttons=[dict(label="Play",
                              method="animate",
                              args=[None])])]))
    fig.show()
                                       
                                       

animate_bar_chart()